<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 41.4 MB/s 
     |████████████████████████████████| 166 kB 68.3 MB/s 
     |████████████████████████████████| 182 kB 71.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 74.8 MB/s 
     |████████████████████████████████| 162 kB 77.4 MB/s 
     |████████████████████████████████| 162 kB 75.8 MB/s 
     |████████████████████████████████| 158 kB 75.6 MB/s 
     |████████████████████████████████| 157 kB 77.8 MB/s 
     |████████████████████████████████| 157 kB 79.5 MB/s 
     |████████████████████████████████| 157 kB 66.9 MB/s 
     |████████████████████████████████| 157 kB 78.7 MB/s 
     |████████████████████████████████| 157 kB 79.9 MB/s 
     |████████████████████████████████| 157 kB 80.0 MB/s 
     |████████████████████████████████| 157 kB 62.4 MB/s 
     |████████████████████████████████| 156 kB 76.8 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [7]:
inception = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg')
model_inception=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  inception,
  #layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
inception.trainable=False
model_inception.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 5)                 10245     
                                                                 
Total params: 21,813,029
Trainable params: 10,245
Non-trainable params: 21,802,784
_________________________________________________________________


In [8]:
model_inception.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [11]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "inceptionv3 with globalavgpool",
)

In [12]:
%%time
epochs=40
history = model_inception.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.0900 - accuracy: 0.5923

wandb: Adding directory to artifact (/content/wandb/run-20221105_081640-2s6g0yh5/files/model-best)... Done. 0.9s


22/22 [==============================] - 306s 10s/step - loss: 1.0900 - accuracy: 0.5923 - val_loss: 0.5399 - val_accuracy: 0.8000
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.4139 - accuracy: 0.8722

wandb: Adding directory to artifact (/content/wandb/run-20221105_081640-2s6g0yh5/files/model-best)... Done. 0.8s


...

22/22 [==============================] - 37s 2s/step - loss: 0.0114 - accuracy: 1.0000 - val_loss: 0.0702 - val_accuracy: 0.9657
Epoch 34/40
22/22 [==============================] - 4s 137ms/step - loss: 0.0116 - accuracy: 1.0000 - val_loss: 0.0721 - val_accuracy: 0.9657
Epoch 35/40
22/22 [==============================] - 4s 137ms/step - loss: 0.0119 - accuracy: 1.0000 - val_loss: 0.0716 - val_accuracy: 0.9600
Epoch 36/40
22/22 [==============================] - 4s 138ms/step - loss: 0.0107 - accuracy: 1.0000 - val_loss: 0.0704 - val_accuracy: 0.9657
Epoch 37/40
22/22 [==============================] - 4s 137ms/step - loss: 0.0102 - accuracy: 1.0000 - val_loss: 0.0713 - val_accuracy: 0.9771
Epoch 38/40
22/22 [==============================] - ETA: 0s - loss: 0.0101 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221105_081640-2s6g0yh5/files/model-best)... Done. 0.9s


22/22 [==============================] - 37s 2s/step - loss: 0.0101 - accuracy: 1.0000 - val_loss: 0.0691 - val_accuracy: 0.9714
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.0090 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221105_081640-2s6g0yh5/files/model-best)... Done. 0.9s


22/22 [==============================] - 40s 2s/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.0624 - val_accuracy: 0.9657
Epoch 40/40
22/22 [==============================] - 4s 136ms/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 0.0687 - val_accuracy: 0.9714
CPU times: user 15min 35s, sys: 47.1 s, total: 16min 22s
Wall time: 22min 16s


In [13]:
wandb.finish()

accuracy,▁▆▇▇████████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇█▇▇██▇█████▇█▇██▇█████
val_loss,█▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,38
best_val_loss,0.0624
epoch,39
loss,0.0077
val_accuracy,0.97143
